# More SQL

Today we'll continue where we left off last time, and then
learn how to use multiple tables (we actually did that once
last time, but then we only combined the result of searches
in different tables -- this time we well search several
tables at once).

As usual we have to tell Jupyter to allow SQL:

In [ ]:
%load_ext sql

And then we import the database for today's lecture:

In [ ]:
%sql sqlite:///lect02.sqlite

## Groups and aggregate functions revisited

To understand `GROUP BY`, lets look at the following query:

In [ ]:
%%sql
SELECT category, COUNT()
FROM   nobel
WHERE  year = 2013
GROUP BY category

To see what happens, we can look at the query:

In [ ]:
%%sql
SELECT year, category, laureate
FROM   nobel
WHERE  year = 2013
ORDER BY category

Here the rows of each category will end up ajacent to each
other, and what the `GROUP BY` command does is to insert an
invisible divider between the groups. If we apply an
aggregate function, such as `COUNT()`, in a table we have
grouped, it will be applied to each group, not to the whole
table. So instead of getting one `COUNT()` for the whole
table (it would be a single value), we get one `COUNT()` for
each group (as above).

If we substitute `category` with `laureate` in the first
line, we get a strange result:

In [ ]:
%%sql
SELECT laureate, COUNT()
FROM   nobel
WHERE  year = 2013
GROUP BY category

The problem is that we only get one row per group in the
output, and that there may be several laureates in each
group -- our query will return one of them in a seemingly
haphazard manner.

It we use `category` in the `SELECT` statement (as in the
first query above), we get a value which we know is the same
for each row in the group (by definition, since that's what
we grouped by).

If we're only interested in those categories with less than
three laureates, we use `HAVING` to select only _groups_
with a given property:

In [ ]:
%%sql
SELECT category, COUNT()
FROM   nobel
WHERE  year = 2013
GROUP BY category
HAVING COUNT() < 3

This corresponds to a `WHERE` statement, but it applies to
groups, not to individual rows (as `WHERE` does).

In the query above we first have a `WHERE` statement to
select some rows from the whole table, and then group the
resulting selection.

We can't have another `WHERE` after the `HAVING`, but we can
make this query a subquery, and have another `WHERE` in the
outer query:

In [ ]:
%%sql
SELECT category, cnt
FROM (
    SELECT category, COUNT() AS cnt
    FROM   nobel
    WHERE  year = 2013
    GROUP BY category
    HAVING COUNT() < 3)
WHERE cnt > 1

**Exercise:** _Has anyone won more than one Nobel price?_

In [ ]:
%%sql


**Exercise:** _Has anyone won more than one Nobel price in the same
category?_

In [ ]:
%%sql


## Subqueries, continued...

Last time we saw a useful form of _subquery_:

```sql
SELECT ...
FROM   ...
WHERE  ... IN
       (SELECT ...
        FROM ...
        WHERE ...)
```


Let's go back to the final problem from last time, and try
to solve it using a subquery.

**Exercise:** _Who has shared the chemistry prize with exactly one
other laureate in years when the summer olympics were held
in Europe?_

In [ ]:
%%sql


In [ ]:
%%sql


In [ ]:
%%sql


## Instead of lists

One of the 'rules' of relational databases is that the
values in our tables must be primitive, we're not allowed to
have lists or objects as values. This requirement is called
["First normal
form"](https://en.wikipedia.org/wiki/First_normal_form), or
"1NF" -- we'll return to 'normal forms' during lectures 7
and 8.

If we were to write a program to keep track of the phone
numbers of our friends, we could write something like this
in Java:

```java
class Friend {

    private String name;
    private List<PhoneNumber> phoneNumbers;
    
    ...
}

List<Friend> friends = ...
```


A literal translation of this into a table would be
something like:

```
name      phone_numbers
----      -------------
Adam      [123456, 654321]
Bodil     [196811]
```


but this is in violation of the 'First normal form', so
instead have a new row for each phone number:

```
name      phone
----      -----
Adam      123456
Adam      654321
Bodil     196811
```


We can ask for Adam's number with:

```sql
SELECT phone
FROM   friends
WHERE  name = 'Adam'
```


and we'd get both numbers back:

```
phone
-----
123456
654321
```


Now, assume we also want to save the birthdays of our
friends. We could add a column for birthdays like this:

```
name    birthday    phone
----    --------    -----
Adam    2 dec       123456
Adam    2 dec       654321
Bodil   30 nov      196811
```


What's the problem with this?

And what if we also want to save our friends' email
addresses:

```
name        birthday    phone       email
----        --------    -----       -----
Adam        2 dec       123456      adam@gmail.com
Adam        2 dec       123456      adam@yahoo.com
Adam        2 dec       654321      adam@gmail.com
Adam        2 dec       654321      adam@yahoo.com
Bodil       30 nov      196811      bodil@itu.dk
Bodil       30 nov      196811      bodil@lu.se
Cecilia     9 apr       511235      cecilia@dn.se
Cecilia     9 apr       641587      cecilia@dn.se
David       12 jun      984531      david@gmail.co
David       12 jun      984531      david@hotmail
Emma        11 aug      123456      emma@lu.se
```


It may seem silly to have four rows for Adam, but if we
removed one of them, we would loose vital information. If we
removed the first row, and someone asked who had the phone
number 123456 and the email address adam@gmail.com, the
query:

```sql
SELECT  name, birthday
FROM    friends
WHERE   phone = '123456' AND email = 'adam@gmail.com'
```


would give an empty result.

The problem with the table above is that we have a lot of
redundance, i.e., information is repeated in many places.
This is not only a waste of storage space, it also makes it
much, much harder to work with our table -- insertions,
updates and removals affects many rows, and if we're not
careful, we might leave the table in a corrupted state.

**Exercise:** _The data in the table above would be much easier to
maintain if we split it into several tables -- suggest how
to do that._


## Joining tables together

The basic idea and data for the following example is taken
from Jennifer Widom, she's one of the authors of the course
book (I've made some name alterations to comply with the
coding conventions I use).

We want a database for handling college applications, a
number of students applies for various majors at different
colleges. We want to keep track of:

 + student id (similar to stil-id)
 + student name
 + student's grade average (gpa)
 + the size of the student's highschool
 + the names of the colleges
 + the state for each college
 + the enrollment for each college
 + the major applied for
 + the decision ('Y' for accepted, 'N' otherwise)

One way of doing this would be to use one big table:

In [ ]:
%%sql
SELECT *
FROM   big_college

But, just as with our friends database above, this would
give us a lot of redundance, so we'd like to split this big
table into several related tables.

In lectures 7 and 8 we'll discuss the theory and practice of
splitting up tables -- for now, suggest tables which you
think would make working with our data easier.

<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>

We have one table for the students:

In [ ]:
%%sql
SELECT *
FROM   students
LIMIT 4

One for the colleges:

In [ ]:
%%sql
SELECT *
FROM   colleges
LIMIT 4

And one for all collage applications made by the students:

In [ ]:
%%sql
SELECT *
FROM   applications
LIMIT 4

The `applications` table uses `s_id` instead of `s_name`,
since we could have several students with the same name (we
actually have two 'Amy') -- `s_id` is guaranteed to be
unique (we'll talk _much_ more about uniqueness next week).

Now assume we want to display all applications, _with the
names of the students_ (not just student id). We then need a
way to combine information in the `applications` table with
information in the `students` table, and we'll do it with a
_join_.


The SQL standard specifies five different kinds of joins:

 + _cross join_
 + _inner join_
 + _left outer join_
 + _right outer join_
 + _full outer join_

The most primitive of these is the cross join -- we can
write it as:

```
SELECT      *
FROM        a 
CROSS JOIN  b
```


Doing this, we get all combinations of rows from each of
table `a` and `b` -- the rows will contain all attributes
from both tables (unless we make a projection).

If we do it with our `applications` and `students` tables we
get:

In [ ]:
%%sql
SELECT      *
FROM        applications 
CROSS JOIN  students

The number of rows in this table is the product of the
number of rows in each table, and most of the rows in the
combined table are totally uninteresting.

The only rows of interest to us are those where the student
id from the `applications` table is the same as the student
id from the `students` table, and we can express that as:

In [ ]:
%%sql
SELECT      *
FROM        applications 
CROSS JOIN  students
WHERE       applications.s_id = students.s_id

or, using aliases, so we don't have to write long table
names:

In [ ]:
%%sql
SELECT      *
FROM        applications AS a 
CROSS JOIN  students AS s
WHERE       a.s_id = s.s_id

We can clean up the output using a projection:

In [ ]:
%%sql
SELECT      s.s_name, a.c_name, a.major
FROM        applications AS a 
CROSS JOIN  students AS s
WHERE       a.s_id = s.s_id

or even

In [ ]:
%%sql
SELECT      s_name AS name, c_name AS college, major
FROM        applications AS a
CROSS JOIN  students AS s
WHERE       a.s_id = s.s_id

(we only need to prefix column names when there would
otherwise be an ambiguity).

There is a shortcut for cross joins, we can just put a comma
between the two tables we want to join:

In [ ]:
%%sql
SELECT s_name AS name, c_name AS college, major
FROM   applications AS a, students AS s
WHERE  a.s_id = s.s_id

This works, but there are much more elegant ways write this
query.

An _inner join_ combine two tables into a new table (it's
not actually saved as a table) -- it does this by creating
combined rows only when rows from the two tables 'match'
each other. In this case it's the `s_id` columns which
should match, so we write:

In [ ]:
%%sql
SELECT      s_name AS name, c_name AS college, major
FROM        applications AS a 
INNER JOIN  students AS s 
ON          a.s_id = s.s_id

Logically it can be seen as the equivalent of a cross join
and then a selection, but the database engine normally use
algorithms which are much faster and requires much less
memory when we define an inner join. Once you get used to
them, inner joins will feel much better to use than cross
joins (although, in theory, the database should try to
optimize all queries, and _might_ find an efficient way to
execute even cross joins).

Inner joins are the default joins, so we can omit the word
`INNER`, and just write `JOIN`:

In [ ]:
%%sql
SELECT  s_name AS name, c_name AS college, major
FROM    applications AS a
JOIN    students AS s
ON      a.s_id = s.s_id

The condition on which we join tables is often called the
_join predicate_, and a join where the join predicate is an
equality test, such as here, is sometimes called an
_equi-join_. There is a special form of join when the
columns we're comparing in an equi-join have the same name:

In [ ]:
%%sql
SELECT  s_name AS name, c_name AS college, major
FROM    applications
JOIN    students
USING   (s_id)

One benefit from using `USING` is that the columns we join
over will not be duplicated, we get only one of them in the
output. 

I'd say this is the preferred way of writing this query.

There is an even simpler way to write it, using what's
called a _natural join_, but it's error-prone, and you're
recommended not to use it. It's written as

In [ ]:
%%sql
SELECT        s_name AS name, c_name AS college, major
FROM          applications
NATURAL JOIN  students

and it joins the tables using an equi-join for all columns
with coinciding names -- _this could create big problems if
there are attributes in the tables which just happen to
coincide without us realizing it!_ If we use the `JOIN` ...
`USING` instead, we explicitly declare on which attributes
to join, so we guard ourselves against accidental column
name collisions.

Observe that we can use selections just as before when we
join tables. 

**Exercise:** _Write a SQL query which shows all applications as
above, but only for students applying for CS at Stanford._

In [ ]:
%%sql


**Exercise:** _Write a SQL query which shows all applications
as above, but also shows what state the college is in._

In [ ]:
%%sql


We can join a table with itself, write a query which finds
all pairs with the same Grade Point Average (GPA):

We can do it either with a cross join:

In [ ]:
%%sql
SELECT s1.s_id, s1.s_name, s1.gpa, s2.s_id, s2.s_name, s2.gpa
FROM   students AS s1, students AS s2
WHERE  s1.gpa = s2.gpa AND
       s1.s_id < s2.s_id

or with an inner join:

In [ ]:
%%sql
SELECT s1.s_id, s1.s_name, s2.s_id, s2.s_name, gpa
FROM   students AS s1 
JOIN   students AS s2 
USING  (gpa)
WHERE  s1.s_id < s2.s_id

Joining a table with itself is called a _self join_.